# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [4]:
import pandas as pd
import json
evidence = pd.read_json('data/evidence.json', orient='index').reset_index().rename(columns={'index': 'evidence_id', 0: 'evidence_text'})
train = pd.read_json('data/train-claims.json', orient='index').reset_index().rename(columns={'index': 'claim_id'})
dev = pd.read_json('data/dev-claims.json', orient='index')
dev_baseline = pd.read_json('data/dev-claims-baseline.json', orient='index')
test = pd.read_json('data/test-claims-unlabelled.json', orient='index')   

# train = train.explode('evidences')
# train = pd.merge(train, evidence, on='evidences', how='left')
train.head(10)
#evidence.head()

,claim_id,claim_text,claim_label,evidences
0,claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1..."
1,claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]"
2,claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]"
3,claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5..."
4,claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72..."
5,claim-851,The last time the planet was even four degrees...,NOT_ENOUGH_INFO,"[evidence-226174, evidence-1049316, evidence-3..."
6,claim-2773,Tree-ring proxy reconstructions are reliable b...,DISPUTED,"[evidence-974673, evidence-602109]"
7,claim-949,"Under the most ambitious scenarios, they found...",DISPUTED,"[evidence-707654, evidence-28478, evidence-491..."
8,claim-1019,An additional kick was supplied by an El Niño ...,NOT_ENOUGH_INFO,"[evidence-863309, evidence-61462, evidence-639..."
9,claim-2834,When stomata-derived CO2 (red) is compared to ...,SUPPORTS,[evidence-439640]


In [6]:
evidence_dict = dict(zip(evidence['evidence_id'], evidence['evidence_text']))



,claim_id,claim_text,claim_label,evidences,evidence_text
0,claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1...",[At very high concentrations (100 times atmosp...
1,claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]",[While ‘climate change’ can be due to natural ...
2,claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]",[There is evidence of reversals in the prevail...
3,claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5...",[There is no convincing scientific evidence th...
4,claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72...","[With average temperature +8.1 °C (47 °F)., Th..."


In [142]:
label_distribution = train['claim_label'].value_counts()

# Display the distribution
print(label_distribution)

claim_label
SUPPORTS           519
NOT_ENOUGH_INFO    386
REFUTES            199
DISPUTED           124
Name: count, dtype: int64


In [125]:
#import pandas as pd
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Stopwords set for faster membership checking
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Tokenize text
    tokens = text.split()
    # Remove stopwords and apply lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Apply the function to a DataFrame column
def preprocess_series(text_series):
    return text_series.apply(preprocess_text)

evidence['preprocessed_text'] = preprocess_series(evidence['evidence_text'])


In [130]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_data(text):  
    tokens = tt.tokenize(text)
    
    processed_tokens = []
    
    for token in tokens:
        token = token.lower()
        if token not in stopwords and token.isalpha():
            lemma = lemmatizer.lemmatize(token)
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)

train['processed_claim_text'] = train['claim_text'].apply(preprocess_data)
train['processed_evidence_text'] = train['evidence_text'].apply(preprocess_data)
evidence['preprocessed_text'] = evidence['evidence_text'].apply(preprocess_data)

train_claims = train['processed_claim_text']
train_evidences = train['processed_evidence_text']

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'all_evidence' is a DataFrame containing all the evidence passages
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(evidence['preprocessed_text'])

In [134]:
def retrieve_documents(query, tfidf_matrix, vectorizer, top_k=5):
    # Transform the query to the same TF-IDF vector space as the evidence
    query_tfidf = vectorizer.transform([query])
    
    # Compute cosine similarities between the query and all documents
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Get the top k indices of documents with the highest similarity scores
    top_k_indices = cosine_similarities.argsort()[-top_k:][::-1]
    
    return top_k_indices



# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*